In [1]:
from parsi_io.modules.number_extractor import NumberExtractor
extractor = NumberExtractor()
res = extractor.run('پانزده بیست و پنجم ')
res[0]['value'] / res[1]['value']

0.6

In [17]:
from parsi_io.modules.number_extractor.pattern import *

import re

txt = 'پانزده بیست و پنجم '
x = re.findall(r'{PATTERN_SEARCH} {PATTERN_SEARCH}', txt)
x

[]

In [4]:
from parsi_io.modules.number_extractor import NumberExtractor
extractor = NumberExtractor()
res = extractor.run(' در یک ده هزارم سالگرد ')
res

[{'span': [4, 15], 'phrase': 'یک ده هزارم', 'value': 0.0001}]

In [3]:
res = extractor.run(' صد و بیست و سه هزار و سیصد و  شصت و چهار ')
res

[{'span': [1, 20], 'phrase': 'صد و بیست و سه هزار', 'value': 123000},
 {'span': [25, 41], 'phrase': 'صد و  شصت و چهار', 'value': 164}]

In [2]:
res = extractor.run(' پنجاه و شش صدم ')
res

[{'span': [1, 6], 'phrase': 'پنجاه', 'value': 50},
 {'span': [9, 14], 'phrase': 'شش صد', 'value': 600}]

In [3]:
res = extractor.run(' بیست و سه هزار و صد و شصت و شش ')
res

[{'span': [1, 31], 'phrase': 'بیست و سه هزار و صد و شصت و شش', 'value': 23166}]

In [16]:
res = extractor.run(' یک صدم ')
res

[{'span': [1, 6], 'phrase': 'یک صد', 'value': 100}]

In [1]:
from __future__ import unicode_literals
from hazm import *


normalizer = Normalizer()
normalizer.normalize('اصلاح نويسه ها و استفاده از نیم‌فاصله پردازش را آسان مي كند')



ModuleNotFoundError: No module named 'hazm'

In [1]:
import re
import unidecode

def join_patterns(patterns, grouping=False):
    return '(' + ('?:' if not grouping else '') + '|'.join(patterns) + ')'

NUM_1_2_to_9_DICT = {
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'سو': 3,
    'چهار': 4,
    'پنج': 5,
    'شش': 6,
    'هفت': 7,
    'هشت': 8,
    'نه': 9
}

NUM_10_11_to_19_DICT = {
    'ده': 10,
    'یازده': 11,
    'دوازده': 12,
    'سیزده': 13,
    'چهارده': 14,
    'پانزده': 15,
    'شانزده': 16,
    'هفده': 17,
    'هجده': 18,
    'نوزده': 19
}

NUM_20_30_to_90_DICT = {
    'بیست': 20,
    'سی': 30,
    'چهل': 40,
    'پنجاه': 50,
    'شصت': 60,
    'هفتاد': 70,
    'هشتاد': 80,
    'نود': 90,
}

NUM_100_200_to_900_DICT = {
    'صد': 100,
    'یکصد': 100,
    'دویست': 200,
    'سیصد': 300,
    'چهارصد': 400,
    'پانصد': 500,
    'ششصد': 600,
    'هفتصد': 700,
    'هشتصد': 800,
    'نهصد': 900
}

NUM_EXTENDER_DICT = {
    'هزار': 1E3,
    'میلیون': 1E6,
    'میلیارد': 1E9
}

NUM_SPECIAL_FRACTION_DICT = {
    'نیم': 1 / 2,
    'نصف': 1 / 2,
    'ثلث': 1 / 3,
    'ربع': 1 / 4,
    'خمس': 1 / 5
}

NUM_UNDER_1000_DICT = {
    **NUM_1_2_to_9_DICT,
    **NUM_10_11_to_19_DICT,
    **NUM_20_30_to_90_DICT,
    **NUM_100_200_to_900_DICT
}

WHITE_SPACE = r'[\s\u200c]+'
CONNECTOR = rf'{WHITE_SPACE}و{WHITE_SPACE}'
NEGATE = r'منفی'
ZERO = r'صفر'
DIGITS = r'[۰۱۲۳۴۵۶۷۸۹0123456789]'
DOTS = r'[٫\.]'

# Pattern for digit-based NUM
NUM_DIGIT_BASED = rf'(?:-?{DIGITS}+(?:{DOTS}{DIGITS}+)?)'

# Pattern for NUM ∈ {1,2,...,99}
NUM_1_2_to_9 = join_patterns(NUM_1_2_to_9_DICT.keys(), True)
NUM_10_11_to_19 = join_patterns(NUM_10_11_to_19_DICT.keys(), True)
NUM_20_30_to_90 = join_patterns(NUM_20_30_to_90_DICT.keys(), True)
NUM_20_21_to_99 = rf'(?:{NUM_20_30_to_90}(?:{CONNECTOR}{NUM_1_2_to_9})?)'
NUM_1_2_to_99 = join_patterns([NUM_10_11_to_19, NUM_20_21_to_99, NUM_1_2_to_9])

# Pattern for NUM ∈ {1,2,...,999}
NUM_100_200_to_900 = join_patterns(NUM_100_200_to_900_DICT.keys(), True)
NUM_100_101_to_999 = rf'(?:{NUM_100_200_to_900}(?:{CONNECTOR}{NUM_1_2_to_99})?)'
NUM_1_2_to_999 = join_patterns([NUM_100_101_to_999, NUM_1_2_to_99, NUM_DIGIT_BASED])

# Pattern for NUM ∈ {1,2,...}
NUM_EXTENDER = join_patterns(NUM_EXTENDER_DICT.keys(), True)
NUM_1_2_to_999_EXTENDED = rf'(?:(?:{NUM_1_2_to_999}{WHITE_SPACE})?{NUM_EXTENDER}(?:{WHITE_SPACE}{NUM_EXTENDER})*)'
NUM_EXTENDED = rf'(?:{NUM_1_2_to_999_EXTENDED}(?:{CONNECTOR}{NUM_1_2_to_999_EXTENDED})*(?:{CONNECTOR}{NUM_1_2_to_999})?)'
NUM_NATURAL = join_patterns([NUM_EXTENDED, NUM_1_2_to_999])

# Pattern for NUM ∈ Z
NUM_INTEGER = join_patterns([ZERO, rf'(?:(?:({NEGATE}){WHITE_SPACE})?{NUM_NATURAL})'])

# Pattern for NUM ∈ {p/q | p,q ∈ Z}
NUM_NORMAL_FRACTION = rf'(?:{NUM_INTEGER}{WHITE_SPACE}{NUM_INTEGER}م)'
NUM_NORMAL_FRACTION_LABELED = rf'(?:(?P<numerator>{NUM_INTEGER}){WHITE_SPACE}(?P<denominator>{NUM_INTEGER})م)'
NUM_SPECIAL_FRACTION = join_patterns(NUM_SPECIAL_FRACTION_DICT.keys())
NUM_FRACTION = join_patterns([NUM_SPECIAL_FRACTION, NUM_NORMAL_FRACTION])

# Pattern for NUM ∈ {n+(p/q) | n,p,q ∈ Z}
NUM_INTEGER_PLUS_FRACTION_LABELED = rf'(?:(?P<integer_part>{NUM_INTEGER}){CONNECTOR}(?P<fraction_part>{NUM_FRACTION}))'
NUM_INTEGER_PLUS_FRACTION = rf'({NUM_INTEGER}{CONNECTOR}{NUM_FRACTION})'

# Pattern for NUM ∈ Q
NUM_RATIONAL = join_patterns([NUM_FRACTION, NUM_INTEGER_PLUS_FRACTION, NUM_INTEGER])

def get_value_NUM_DIGIT_BASED(num_string):
    return float(unidecode.unidecode(num_string))

def get_value_NUM_1_2_to_999(num_string):
    try:
        return get_value_NUM_DIGIT_BASED(num_string)
    except:
        match_groups = re.match(NUM_1_2_to_999, num_string).groups()
        return sum([NUM_UNDER_1000_DICT[key] for key in filter(None, match_groups)])

def get_value_NUM_1_2_to_999_EXTENDED(num_string):
    extender_value = 1
    num_extender_matches = re.finditer(NUM_EXTENDER, num_string)
    for match in num_extender_matches:
        extender_value *= NUM_EXTENDER_DICT[match.group()]
    try:
        num_without_extender_match = re.search(NUM_1_2_to_999, re.sub(NUM_EXTENDER, '', num_string))
        num_without_extender = get_value_NUM_1_2_to_999(num_without_extender_match.group())
    except:
        num_without_extender = 1
    return extender_value * num_without_extender

def get_value_NUM_NATURAL(num_string):
    num_1_2_to_999_extended_list = re.finditer(NUM_1_2_to_999_EXTENDED, num_string)
    num_value = 0
    for match in num_1_2_to_999_extended_list:
        num_value += get_value_NUM_1_2_to_999_EXTENDED(match.group())
    num_1_2_to_999_at_the_end = re.search(NUM_1_2_to_999, re.sub(NUM_1_2_to_999_EXTENDED, '', num_string))
    if num_1_2_to_999_at_the_end:
        num_value += get_value_NUM_1_2_to_999(num_1_2_to_999_at_the_end.group())
    return num_value

def get_value_NUM_INTEGER(num_string):
    if re.search(ZERO, num_string):
        return 0
    coefficient = 1
    if re.search(NEGATE, num_string):
        coefficient = -1
    num_natural = re.search(NUM_NATURAL, num_string).group()
    return coefficient * get_value_NUM_NATURAL(num_natural)

def get_value_NUM_FRACTION(num_string):
    match = re.search(NUM_SPECIAL_FRACTION, num_string)
    if match:
        return NUM_SPECIAL_FRACTION_DICT[match.group()]
    match = re.search(NUM_NORMAL_FRACTION_LABELED, num_string)
    numerator = get_value_NUM_INTEGER(match.group('numerator'))
    denominator = get_value_NUM_INTEGER(match.group('denominator'))
    try:
        return numerator / denominator
    except:
        return 0

def get_value_NUM_INTEGER_PLUS_FRACTION(num_string):
    match = re.search(NUM_INTEGER_PLUS_FRACTION_LABELED, num_string)
    integer_part = get_value_NUM_INTEGER(match.group('integer_part'))
    try:
        fraction_part = get_value_NUM_FRACTION(match.group('fraction_part'))
    except:
        fraction_part = 0
    return integer_part + fraction_part

def get_value_NUM_RATIONAL(num_string):
    NUM_FRACTION_match = re.fullmatch(NUM_FRACTION, num_string)
    if NUM_FRACTION_match:
        return get_value_NUM_FRACTION(num_string)
    NUM_INTEGER_PLUS_FRACTION_match = re.fullmatch(NUM_INTEGER_PLUS_FRACTION, num_string)
    if NUM_INTEGER_PLUS_FRACTION_match:
        return get_value_NUM_INTEGER_PLUS_FRACTION(num_string)
    return get_value_NUM_INTEGER(num_string)

def extract_numbers(text):
    extracted_numbers = []
    for match in re.finditer(NUM_RATIONAL, text):
        extracted_numbers.append({
            'marker': match.group(),
            'span': match.span(),
            'value': get_value_NUM_RATIONAL(match.group())
        })
    return extracted_numbers

print(extract_numbers("یی، یک دو سال"))

[{'marker': 'یک', 'span': (4, 6), 'value': 1}, {'marker': 'دو', 'span': (7, 9), 'value': 2}]


In [9]:
text = ' هشتاد و پنج صدم '
extract_numbers(text)

[{'marker': '0.85', 'span': (1, 5), 'value': 0.85}]

In [29]:
dict_units = {'MP': 'megapixels',
 'Pa': 'Pa',
 'bu': 'bushels',
 'dpcm': 'dpcm',
 'dpi': 'dpi',
 'em': 'em',
 'fm': 'fathoms',
 'fur': 'furlongs',
 'ppcm': 'ppcm',
 'ppi': 'ppi',
 'px': 'pixels',
 'st': 'stones',
 '°': '°',
 'آمپر': 'amps',
 'اتمسفر': 'atm',
 'اسب بخار': 'hp',
 'الکترون\u200cولت': 'electronvolt',
 'اهم': 'ohms',
 'اونس': 'oz',
 'اونس تروا': 'oz troy',
 'اونس سیال': 'fl oz',
 'اونس سیال انگلیسی': 'Imp. fl oz',
 'اینچ': 'inches',
 'اینچ جیوه': 'inHg',
 'اینچ مربع': 'inches²',
 'اینچ مکعب': 'inches³',
 'بار': 'bar',
 'بایت': 'byte',
 'بخش/میلیون': 'parts/million',
 'بشکه': 'barrel',
 'بیت': 'bit',
 'بی\u200cتی\u200cیو': 'BTU',
 'تابندگی خورشید': 'solar luminosities',
 'ترابایت': 'TByte',
 'ترابیت': 'Tbit',
 'ترم آمریکایی': 'US therm',
 'تن': 'tons',
 'تن متریک': 't',
 'ثانیه': 'secs',
 'ثانیهٔ قوسی': 'arcsecs',
 'جرم خورشید': 'solar masses',
 'جرم زمین': 'Earth masses',
 'جریب': 'acres',
 'جریب فوت': 'acre ft',
 'دالتون': 'daltons',
 'درجه': 'degrees',
 'درجهٔ سلسیوس': 'deg. C',
 'درجهٔ فارنهایت': 'deg. F',
 'درجهٔ کلوین': 'K',
 'درصد': 'percent',
 'دسی\u200cلیتر': 'dL',
 'دسی\u200cمتر': 'dm',
 'دقیقه': 'mins',
 'دقیقهٔ قوسی': 'arcmins',
 'دهه': 'dec',
 'ده\u200cهزارم': 'permyriad',
 'دور': 'rev',
 'دونوم': 'dunams',
 'رادیان': 'radians',
 'روز': 'days',
 'ساعت': 'hours',
 'سال': 'years',
 'سال نوری': 'light yrs',
 'سانت': 'cm',
 'سانتی\u200cلیتر': 'cL',
 'سانتی\u200cمتر مربع': 'cm²',
 'سانتی\u200cمتر مکعب': 'cm³',
 'سده': 'c',
 'شعاع خورشید': 'solar radii',
 'عیار': 'karats',
 'فوت': 'feet',
 'فوت مربع': 'sq feet',
 'فوت مکعب': 'feet³',
 'قاشق غ.': 'tbsp',
 'قاشق چ.': 'tsp',
 'قیراط': 'carats',
 'لوکس': 'lux',
 'لیتر': 'liters',
 'لیتر/کیلومتر': 'liters/km',
 'لیتر/۱۰۰ کیلومتر': 'L/100 km',
 'ماه': 'months',
 'مایل': 'miles',
 'مایل اسکاندیناوی': 'smi',
 'مایل در ساعت': 'miles/hour',
 'مایل دریایی': 'nmi',
 'مایل مربع': 'sq miles',
 'مایل مکعب': 'mi³',
 'مایل/امپریال گالن': 'miles/gal Imp.',
 'مایل/گالن': 'miles/gal',
 'متر': 'm',
 'متر در ثانیه': 'meters/sec',
 'متر مربع': 'meters²',
 'متر مکعب': 'm³',
 'متر/مجذور ثانیه': 'meters/sec²',
 'مول': 'mole',
 'مگابایت': 'MByte',
 'مگابیت': 'Mbit',
 'مگالیتر': 'ML',
 'مگاهرتز': 'MHz',
 'مگاوات': 'MW',
 'مگاپاسکال': 'MPa',
 'میلی\u200cآمپر': 'milliamps',
 'میلی\u200cبار': 'mbar',
 'میلی\u200cثانیه': 'millisecs',
 'میلی\u200cلیتر': 'mL',
 'میلی\u200cمتر جیوه': 'mmHg',
 'میلی\u200cمول/لیتر': 'millimol/liter',
 'میلی\u200cوات': 'mW',
 'میلی\u200cگرم': 'mg',
 'میلی\u200cگرم در دسی\u200cلیتر': 'mg/dL',
 'میکروثانیه': 'µsecs',
 'میکرومتر': 'µmeters',
 'میکروگرم': 'µg',
 'م\u200cم': 'mm',
 'نانوثانیه': 'nanosecs',
 'نانومتر': 'nm',
 'نیروی جاذبه': 'g-force',
 'نیوتن': 'newton',
 'نیوتن\u200cمتر': 'N⋅m',
 'هرتز': 'Hz',
 'هزارم': 'permille',
 'هفته': 'weeks',
 'هکتار': 'hectares',
 'هکتولیتر': 'hL',
 'هکتوپاسکال': 'hPa',
 'وات': 'watts',
 'واحد نجومی': 'au',
 'ولت': 'volts',
 'پارسک': 'parsecs',
 'پاینت': 'pints',
 'پاینت متریک': 'mpt',
 'پتابایت': 'PByte',
 'پونت': 'points',
 'پوند': 'pounds',
 'پوند در اینچ مربع': 'psi',
 'پوند-فوت': 'lbf⋅ft',
 'پوند-نیرو': 'pound-force',
 'پیمانه': 'cups',
 'پیمانهٔ متریک': 'mcup',
 'پیکومتر': 'pm',
 'ژول': 'joules',
 'ک.هرتز': 'kHz',
 'ک.وات': 'kW',
 'ک.وات\u200cساعت': 'kW-hour',
 'ک.پاسکال': 'kPa',
 'ک.ژول': 'kilojoule',
 'ک.کالری': 'kcal',
 'کالری': 'Cal',
 'کوارت': 'qts',
 'کیلوبایت': 'kByte',
 'کیلوبیت': 'kbit',
 'کیلومتر': 'km',
 'کیلومتر در ساعت': 'km/hour',
 'کیلومتر مربع': 'km²',
 'کیلومتر مکعب': 'km³',
 'کیلوگرم': 'kg',
 'گالن': 'gal',
 'گالن امپریال': 'Imp. gal',
 'گرم': 'g',
 'گره': 'kn',
 'گیگابایت': 'GByte',
 'گیگابیت': 'Gbit',
 'گیگاهرتز': 'GHz',
 'گیگاوات': 'GW',
 'یارد': 'yards',
 'یارد مربع': 'yards²',
 'یارد مکعب': 'yards³',
 'نیوتون': 'newton'}

In [30]:
dict_prefix = {
    'کیلو': 'k',
    'مگا': 'M',
}

In [38]:
import re
import pint

class ComplexUnit:

    def __init__(self):
        self.numerator_units = list()
        self.denominator_units = list()
        self.dimensionality = None


class Unit:

    def __init__(self, name, prefix, dimensionality):
        self.name = name
        self.prefix = prefix
        self.dimensionality = dimensionality


def join_patterns(patterns, grouping=False):
    return '(' + ('?:' if not grouping else '') + '|'.join(patterns) + ')'


WHITE_SPACE = r'[\s\u200c]+'

UNIT = join_patterns(['گرم', 'متر', 'نیوتون', 'وات', 'm', 'h'], True)  # TODO: A function to get all units
PREF = join_patterns(['کیلو', 'سانتی', 'دسی', 'مگا', 'k'], True)  # TODO: A function to get all prefixes
UNIT_PREF = rf'(?:(?:{PREF}(?:{WHITE_SPACE})?)?{UNIT})'
UNIT_PREF_DIM = rf'(({UNIT_PREF}({WHITE_SPACE}(مربع|مکعب))?)|(((مربع|مکعب|مجذور){WHITE_SPACE})?{UNIT_PREF}))'
MULT_CONNECTOR = join_patterns([rf'({WHITE_SPACE}(در{WHITE_SPACE})?)', rf'(({WHITE_SPACE})?(\*|×)({WHITE_SPACE})?)'])
UNIT_PREF_DIM_SEQ = rf'({UNIT_PREF_DIM}({MULT_CONNECTOR}{UNIT_PREF_DIM})*)'
DIV_CONNECTOR = join_patterns([rf'({WHITE_SPACE}بر{WHITE_SPACE})', rf'(({WHITE_SPACE})?(\/|÷)({WHITE_SPACE})?)'])
COMPLEX_UNIT = rf'({UNIT_PREF_DIM_SEQ}({DIV_CONNECTOR}{UNIT_PREF_DIM_SEQ})*)'

def sing_unit_to_str(unit):
    print(unit.prefix, unit.name)
    pr = '' if unit.prefix is None else dict_prefix[unit.prefix]
    print(unit.dimensionality)
    dim = 1 if unit.dimensionality is None else int(unit.dimensionality.split('**')[1])
    return  f'({pr}{dict_units[unit.name]}**{dim})'

def unit_to_str(complex_unit):
    num = '(' + '*'.join([sing_unit_to_str(unit) for unit in complex_unit.numerator_units]) + ')'
    den = '(' + '*'.join([sing_unit_to_str(unit) for unit in complex_unit.denominator_units]) + ')'
    return num + '/' + den


def extract_units(text):
    def getComplexUnitInstance(text, level):
        pattern = [COMPLEX_UNIT, UNIT_PREF_DIM_SEQ, UNIT_PREF_DIM, UNIT_PREF][level]
        matches = [match for match in re.finditer(pattern, text)]
        if level == 0:
            print("level 0", matches)
            return [{
                'marker': match.group(),
                'span': match.span(),
                'object': getComplexUnitInstance(match.group(), 1)
            } for match in matches]
        if level == 1:
            complexUnit = ComplexUnit()
            complexUnit.numerator_units = getComplexUnitInstance(matches[0].group(), 2)
            if len(matches) > 1:
                for match in matches[1:]:
                    for unit in getComplexUnitInstance(match.group(), 2):
                        complexUnit.denominator_units.append(unit)
            return complexUnit
        if level == 2:
            return [getComplexUnitInstance(match.group(), 3) for match in matches]
        if level == 3:
            power = 1
            if re.findall("مجذور|مربع", text):
                power = 2
            if re.findall("مکعب", text):
                power = 3
            prefix, name = tuple(re.search(UNIT_PREF, matches[0].group()).groups())
            dimensionality = f"[length] ** {power}"
            return Unit(name, prefix, dimensionality)

    return getComplexUnitInstance(text, 0)


a = extract_units('کیلوگرم متر مربع بر مجذور مگا نیوتون')
ureg = pint.UnitRegistry()
b = ureg(unit_to_str(a[0]['object']))
b

level 0 [<re.Match object; span=(0, 36), match='کیلوگرم متر مربع بر مجذور مگا نیوتون'>]
کیلو گرم
[length] ** 1
None متر
[length] ** 2
مگا نیوتون
[length] ** 2


1.0 <Unit('kilogram * meter ** 2 / meganewton ** 2')>

In [32]:
int('  2')

2